In [1]:
import brightway2 as bw
import bw2io
import bw2regional as bwr
import mrio_common_metadata
import bw_migrations

In [2]:
bw.projects.set_current("EXIOBASE regionalized case study")

# Extracting data

In [3]:
from bw2io.extractors import ExcelExtractor
from copy import deepcopy
from bw2io.importers.base_lci import LCIImporter

In [4]:
foreground_direct = ExcelExtractor.extract("data/Foreground direct.xlsx")

assert len(foreground_direct) == 1
assert foreground_direct[0][0] == 'Sheet1'

foreground_direct = foreground_direct[0][1]

In [5]:
# Remove empty columns
len(foreground_direct), len([x for x in foreground_direct[0] if x])

(115, 30)

In [6]:
def chop_empty_columns(data):
    last_column = max([i for i, value in enumerate(data[0]) if value.strip()])
    return [row[:last_column + 1] for row in data]

In [7]:
foreground_direct = chop_empty_columns(foreground_direct)
len(foreground_direct), len(foreground_direct[0])

(115, 32)

In [8]:
foreground_intermediate = ExcelExtractor.extract("data/Foreground intermediate.xlsx")

assert len(foreground_intermediate) == 1
assert foreground_intermediate[0][0] == 'Sheet1'

foreground_intermediate = foreground_intermediate[0][1]

In [9]:
foreground_intermediate = chop_empty_columns(foreground_intermediate)

In [10]:
def transform_into_datasets(data, col_offset, headers):
    activities = [{'exchanges': []} for _ in data[0][col_offset:]]
    for row_index in range(headers):
        label = data[row_index][col_offset - 1]
        if not label:
            continue
        for col_index, value in enumerate(data[row_index][col_offset:]):
            activities[col_index][label] = value
            
    exc_labels = data[headers - 1][:col_offset]
            
    for row_index in range(headers, len(data)):
        exc_metadata = {label: value for label, value in zip(exc_labels, data[row_index])}
        
        for col_index, value in enumerate(data[row_index][col_offset:]):
            if value:
                dct = deepcopy(exc_metadata)
                dct['amount'] = float(value)
                activities[col_index]['exchanges'].append(dct)

    # Production exchange amounts are given in row 2
    for col_index, value in enumerate(data[2][col_offset:]):
        activities[col_index]['exchanges'].append({'type': 'production', 'amount': float(value)})
                
    return activities

In [11]:
fd = transform_into_datasets(foreground_direct, 4, 7)
fd[0]

{'exchanges': [{'Compartment': '',
   'Product/Substance': 'Link to: Market for arable land {GLO}',
   'Sub-compartment': '',
   'Type': 'ha a',
   'amount': 0.89},
  {'Compartment': '',
   'Product/Substance': 'Link to: Road transport {UA} 16-32 t truck',
   'Sub-compartment': '',
   'Type': 'tkm',
   'amount': 119.17193191325892},
  {'Compartment': '',
   'Product/Substance': 'Link to: N-fertiliser, urea, as N {UA}',
   'Sub-compartment': '',
   'Type': 'kg',
   'amount': 8.763862591290234},
  {'Compartment': '',
   'Product/Substance': 'Link to: N-fertiliser, ammonium nitrate, as N {UA}',
   'Sub-compartment': '',
   'Type': 'kg',
   'amount': 49.49959426562078},
  {'Compartment': '',
   'Product/Substance': 'Link to: N-fertiliser, ammonium sulphate, as N {UA}',
   'Sub-compartment': '',
   'Type': 'kg',
   'amount': 1.736543143088991},
  {'Compartment': '',
   'Product/Substance': 'Link to: P-fertiliser, triple superphosphate, as P2O5 {UA}',
   'Sub-compartment': '',
   'Type': 'kg

In [12]:
fd[-1]

{'exchanges': [{'type': 'production', 'amount': 1.0}],
 'Product': 'Treatment of shells',
 'Unit': 'kg',
 'Category': '\\_5 Veg Oil case\\_food industry',
 'Comment': '',
 'Type': 'Material'}

In [13]:
fi = transform_into_datasets(foreground_intermediate, 4, 7)
fi[0]

{'exchanges': [{'Compartment': '',
   'Product/Substance': '_61 N-fertiliser {WL} (product market, hybrid units)',
   'Sub-compartment': '',
   'Type': 'ton',
   'amount': 1.0},
  {'type': 'production', 'amount': 1.0}],
 'Product': 'Link to: N-fertiliser, ammonium nitrate, as N {AR}',
 'Unit': 'ton',
 'Category': '\\_5 Veg Oil case\\_Link to background: Exiobase',
 'Comment': 'Link to Exiobase v3.3.13b2',
 'Type': 'Material'}

In [14]:
fi[-1]

{'exchanges': [{'Compartment': '',
   'Product/Substance': 'Arable land, as ha*year-eq. (linked)',
   'Sub-compartment': '',
   'Type': 'ha a',
   'amount': 1.0},
  {'type': 'production', 'amount': 1.0}],
 'Product': 'Link to: Market for arable land {GLO}',
 'Unit': 'ha a',
 'Category': '\\_5 Veg Oil case\\_Link to background: Exiobase',
 'Comment': 'Link to Exiobase v3.3.13b2. This activity is empty because the refinery capital goods and services are already included with the oil mill capital goods and services (because the oil mill and refinery are aggregated in Exiobase)',
 'Type': 'Material'}

# Creating an `importer` object

In [15]:
ei = LCIImporter("Oil seeds")
ei.data = fd + fi
del ei.strategies[2]

Now the fun part begins. We need to get things into a format the Brightway can understand, and one where it can find links. Right now we have lot of exchanges, but no links. Let's start with standardizing some labels.

## Standard labels

In [16]:
def change_activity_labels(data):
    mapping = {
        'Product': 'name',
        'Type': 'kind',
        'Unit': 'unit'
    }
    for ds in data:
        for key, value in mapping.items():
            if key in ds:
                ds[value] = ds.pop(key)
    return data

In [17]:
def change_exchange_labels(data):
    mapping = {
        'Product/Substance': 'name',
        'Type': 'unit',  # !?
    }
    for ds in data:
        for exc in ds['exchanges']:
            for key, value in mapping.items():
                if key in exc:
                    exc[value] = exc.pop(key)
    return data

In [18]:
ei.apply_strategy(change_activity_labels)
ei.apply_strategy(change_exchange_labels)

Applying strategy: change_activity_labels
Applying strategy: change_exchange_labels


## `unit`

In [19]:
from bw2io.strategies import normalize_units

In [20]:
ei.apply_strategy(normalize_units)

Applying strategy: normalize_units


We have one more unit to fix - `ha a`, or the occupation of one hectare for one year. Normally we could do `ei.migrate("default-units")`, which would convert this unit to what our base flow list expects (occupation of square meter - year), but this would change all our tonnes to kilograms, meaning we would have the wrong unit to link to exiobase. So we treat this as a special case migration.

('ha a', 'square meter-year', 1e4)

In [21]:
bw.Migration("hecatare-units").write({
    'fields': ['unit'],
    'data': [
        (
            ('ha a',),
            {'unit': 'square meter-year', 'multiplier': 1e4}
        )
    ]
}, 'Change only `ha a` units')

ei.migrate("hecatare-units")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [22]:
{ds['unit'] for ds in ei.data}.union({exc.get('unit') for ds in ei.data for exc in ds['exchanges']})

{'MEUR2011',
 None,
 'cubic meter',
 'kilogram',
 'kilowatt hour',
 'megajoule',
 'square meter-year',
 'ton',
 'ton kilometer'}

EXIOBASE uses `Meuro` instead of `MEUR2011`, let's change this.

In [23]:
bw.Migration("MEUR2011").write({
    'fields': ['unit'],
    'data': [
        (
            # First element is input data in the order of `fields` above
            ('MEUR2011',),
            # Second element is new values to substitute
            {
                'unit': 'Meuro',
            }
        )
    ]
}, 'Change Euro unit label')

ei.migrate("MEUR2011")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [24]:
{ds['unit'] for ds in bw.Database("EXIOBASE 3.3.17 hybrid")}

{'Meuro', 'TJ', 'ton'}

## `database`

We can label each dataset with out chosen database name

In [25]:
from bw2io.strategies import add_database_name
from functools import partial

In [26]:
ei.apply_strategy(partial(add_database_name, name=ei.db_name))

Applying strategy: add_database_name


## `categories` and exchange types

Let's fix up this `Compartment` and `Sub-compartment` stuff to our standard `categories`:

In [27]:
def relabel_categories(data):
    for ds in data:
        for exc in ds['exchanges']:
            if 'Compartment' in exc:
                exc['categories'] = (exc.pop("Compartment").lower(), exc.pop("Sub-compartment").lower())
    return data

In [28]:
ei.apply_strategy(relabel_categories)

Applying strategy: relabel_categories


In [29]:
from bw2io.strategies import drop_unspecified_subcategories

In [30]:
ei.apply_strategy(drop_unspecified_subcategories)

Applying strategy: drop_unspecified_subcategories


I think every exchange with `categories` is a biosphere exchange, let's check.

In [31]:
{exc['name'] for ds in ei.data for exc in ds['exchanges'] if exc.get('categories')}

{'Ammonia',
 'Carbon dioxide',
 'Dinitrogen monoxide',
 'Hexane',
 'Hydrogen sulfide',
 'Methane',
 'Nitrate',
 'Nitrogen oxides',
 'Occupation, arable',
 'Phosphorus'}

Looks good. Let's mark these as `biosphere` exchanges, and the others as `technosphere` exchanges.

In [32]:
def label_exchange_type(data):
    for ds in data:
        for exc in ds['exchanges']:
            if exc.get('type'):
                # production exchanges from initial import
                continue
            exc['type'] = 'biosphere' if exc.get('categories') else 'technosphere'
    return data

In [33]:
ei.apply_strategy(label_exchange_type)

Applying strategy: label_exchange_type


## `location`

We will need to strip locations from names like `Market for arable land {GLO}`. We can do this with a regular expression:

In [34]:
import re

In [35]:
location_finder = re.compile(r"\{(.*?)\}")

In [36]:
location_finder.findall("Market for arable land {GLO}")

['GLO']

In [37]:
def strip_locations(data):
    for ds in data:
        location = location_finder.findall(ds['name'])
        if location:
            ds['name'] = ds['name'].replace("{" + location[0] + "}", "").strip()
            ds['location'] = location[0]
        
        for exc in ds['exchanges']:
            if 'name' not in exc:
                continue
            location = location_finder.findall(exc['name'])
            if location:
                exc['name'] = exc['name'].replace("{" + location[0] + "}", "").strip()
                exc['location'] = location[0]
    return data

In [38]:
ei.apply_strategy(strip_locations)

Applying strategy: strip_locations


In [39]:
{ds['name'] for ds in ei.data}

{'Barley cultivation [barley]/UA',
 'Barley to generic market for feed [energy feed]/GLO',
 'Link to: 2.4  Dimethyl amine 40% as A.I.',
 'Link to: Agricultural machinery',
 'Link to: Ammonium glufosinat (Inteam 150 SL)',
 'Link to: Azoxistrobin dan Difenokonazo (Amnistra top)',
 'Link to: Benomil',
 'Link to: Bleaching earth',
 'Link to: Caustic Soda, as 100% conc  Market',
 'Link to: Citric acid  Market',
 'Link to: Coal  Fuel',
 'Link to: Cypermethrin',
 'Link to: Diesel  Fuel',
 'Link to: Dimehipo (Marathon 500SL)',
 'Link to: Diuron',
 'Link to: Electricity  Market',
 'Link to: Erkafuron',
 'Link to: Floroksifir metil heptil ester',
 'Link to: Fuel oil  Fuel',
 'Link to: Glufosinate',
 'Link to: Glyphosate, 41% as A.I.',
 'Link to: Hydrogen  Market',
 'Link to: Indaziflam 500 g/l (Becano 500 SC)',
 'Link to: Isopropil amina glyphosate',
 'Link to: Jetfuel  Fuel',
 'Link to: K-fertiliser, pottasium chloride, as K2O',
 'Link to: Karbosulfan 5%  (Marshall 5g)',
 'Link to: Landfill of 

Looks like we need another strategy for pesky name/location combos

In [40]:
location_finder2 = re.compile(r"(?P<geo>/[A-Z0-9]{2,4})$")

In [41]:
location_finder2.findall('Utilisation of sunflower meal as feed [RSM treatment]/GLO')

['/GLO']

In [42]:
location_finder2.findall('Rapeseed oil refinery [Refined rapeseed oil]/EU27')

['/EU27']

In [43]:
def strip_locations2(data):
    for ds in data:
        location = location_finder2.findall(ds['name'])
        if location:
            ds['name'] = ds['name'].replace(location[0], "").strip()
            ds['location'] = location[0][1:]
        
        for exc in ds['exchanges']:
            if 'name' not in exc:
                continue
            location = location_finder2.findall(exc['name'])
            if location:
                exc['name'] = exc['name'].replace(location[0], "").strip()
                exc['location'] = location[0][1:]
    return data

In [44]:
ei.apply_strategy(strip_locations2)

Applying strategy: strip_locations2


There is still one activity without a location:

In [45]:
for ds in ei.data:
    if not ds.get('location'):
        print(ds['name'])

Treatment of shells


In [46]:
def default_global_location(data):
    for ds in data:
        if not ds.get('location') or ds['location'] == '(unknown)':
            ds['location'] = 'GLO'
        for exc in ds['exchanges']:
            if not exc.get('location') or exc['location'] == '(unknown)':
                exc['location'] = 'GLO'

    return data

In [47]:
ei.apply_strategy(default_global_location)

Applying strategy: default_global_location


## `code`

Let's see if we can create a meaningful `code` instead of a random string.

In [48]:
len(ei.data), len({(ds['name'], ds['location']) for ds in ei.data})

(267, 267)

In [49]:
def generate_code(data):
    for ds in data:
        ds['code'] = "{}-{}".format(ds['name'], ds['location'])
    return data

In [50]:
ei.apply_strategy(generate_code)

Applying strategy: generate_code


WIth this code, we can set `input` for the production exchanges

In [51]:
def link_production_exchanges(data):
    for ds in data:
        prod = [exc for exc in ds['exchanges'] if exc['type'] == 'production']
        assert len(prod) == 1
        prod = prod[0]
        prod['input'] = (ds['database'], ds['code'])
    return data

In [52]:
ei.apply_strategy(link_production_exchanges)

Applying strategy: link_production_exchanges


# Internal linking 

We can now apply the "generic" default strategies.

In [53]:
ei.apply_strategies()

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: strip_biosphere_exc_locations
Applied 3 strategies in 0.00 seconds


We are ready to start thinking about internal linking. Let's see if the names will match up, first by checking whether names are unique, and then by looking at them manually

In [54]:
len(ei.data), len({ds['name'] for ds in ei.data})

(267, 92)

In [55]:
dataset_names = {ds['name'] for ds in ei.data}
exchange_names = {exc.get('name') for ds in ei.data for exc in ds['exchanges']}
len(dataset_names), len(exchange_names), len(dataset_names.intersection(exchange_names))

(92, 73, 43)

In [56]:
ei.match_database(fields=['name', 'location', 'unit'])

Applying strategy: link_iterable_by_fields


In [57]:
ei.statistics()

267 datasets
911 exchanges
529 unlinked exchanges
  Type biosphere: 10 unique unlinked exchanges
  Type technosphere: 141 unique unlinked exchanges


(267, 911, 529)

## `biosphere` exchanges

How many can we link without changing anything?

In [58]:
ei.match_database("biosphere3", fields=['name', 'categories', 'unit'])

Applying strategy: link_iterable_by_fields


In [59]:
ei.statistics()

267 datasets
911 exchanges
499 unlinked exchanges
  Type biosphere: 3 unique unlinked exchanges
  Type technosphere: 141 unique unlinked exchanges


(267, 911, 499)

Sweet, only 3 to fix manually. Let's look at them.

In [60]:
ei.write_excel(only_unlinked=True)

Wrote matching file to:
/Users/cmutel/Library/Application Support/Brightway3/EXIOBASE-regionalized-case-study.9666a926e52c1cc772cb07068bf407f2/output/db-matching-Oil-seeds-unlinked.xlsx


`Carbon dioxide` and `Methane`, both of which need to be labeled fossil (or not) to get a match. `Occupation, arable` is a shortened version of what we have in our master flow list:

In [61]:
[x for x in bw.Database("biosphere3") if x['name'].lower().startswith('occupation, arable')]

['Occupation, arable, reduced tillage (obsolete)' (square meter, None, ('natural resource', 'land')),
 'Occupation, arable, conservation tillage (obsolete)' (square meter, None, ('natural resource', 'land')),
 'Occupation, arable land, unspecified use' (square meter-year, None, ('natural resource', 'land')),
 'Occupation, arable, conventional tillage (obsolete)' (square meter, None, ('natural resource', 'land'))]

In [62]:
bw.Migration("oil-bio").write({
    'fields': ['name'],
    'data': [
        (('Carbon dioxide',), {'name': 'Carbon dioxide, fossil',}),
        (('Methane',), {'name': 'Methane, fossil',}),
        (('occupation, arable',), 
         {'name': 'Occupation, arable land, unspecified use', 'categories': ('natural resource', 'land')}),
    ]
}, 'Change some biosphere flows in oil seeds foreground')

ei.migrate("oil-bio")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [63]:
ei.match_database("biosphere3", fields=['name', 'categories', 'unit'])

Applying strategy: link_iterable_by_fields


In [64]:
ei.statistics()

267 datasets
911 exchanges
491 unlinked exchanges
  Type technosphere: 141 unique unlinked exchanges


(267, 911, 491)

This isn't actually correct - there are still unlinked biosphere exchanges, they are just mislabelled for now :)

## `name`

Let's clean up some names, starting with this pattern: `_64 Manufacture of rubber and plastic products (25)  (product market, hybrid units)`.

In [65]:
test_string = '_64 Manufacture of rubber and plastic products (25)  (product market, hybrid units)'

In [66]:
numeric_start = re.compile("^[0-9_]\d\d")

In [67]:
numeric_start.findall(test_string)

['_64']

In [68]:
test_string = '_64 Manufacture of rubber and plastic products (25)'

In [69]:
numeric_end = re.compile("\(\d\d\)$")

In [70]:
numeric_end.findall(test_string)

['(25)']

In [71]:
def clean_name(name):
    name = name.replace("(product market, hybrid units)", "").replace("Link to: ", "").strip()
    prefix = numeric_start.findall(name)
    if prefix:
        name = name.replace(prefix[0], '').strip()
    suffix = numeric_end.findall(name)
    if suffix:
        name = name.replace(suffix[0], '').strip()
    return name

def clean_names(data):
    for ds in data:
        ds['name'] = clean_name(ds['name'])
        for exc in ds['exchanges']:
            if exc.get('name'):
                exc['name'] = clean_name(exc['name'])
    return data

In [72]:
ei.apply_strategy(clean_names)

Applying strategy: clean_names


For whatever reason, they also call one exchange differently depending on whether it is an activity or an exchange reference (`Electricity  Market`, with two spaces, versus `Electricity mix`). Standardize on `Electricity mix`.

In [73]:
bw.Migration("what-is-electricity").write({
    'fields': ['name'],
    'data': [
        (('Electricity  Market',), {'name': 'Electricity mix',}),
    ]
}, "Let's try being consistent, just to see how it feels")

ei.migrate("what-is-electricity")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [74]:
ei.match_database(fields=['name', 'location', 'unit'])

Applying strategy: link_iterable_by_fields


## Linking against EXIOBASE

First try, don't expect great success.

In [75]:
ei.match_database("EXIOBASE 3.3.17 hybrid", fields=['name', 'location', 'unit'])

Applying strategy: link_iterable_by_fields


In [76]:
ei.statistics()

267 datasets
911 exchanges
54 unlinked exchanges
  Type technosphere: 38 unique unlinked exchanges


(267, 911, 54)

In [77]:
ei.write_excel(only_unlinked=True)

Wrote matching file to:
/Users/cmutel/Library/Application Support/Brightway3/EXIOBASE-regionalized-case-study.9666a926e52c1cc772cb07068bf407f2/output/db-matching-Oil-seeds-unlinked.xlsx


## Specific issues

The flow `Arable land, as ha*year-eq.` and the activity `Link to: Market for arable land {GLO}` are removed, as we don't know where they occur, and are included to use with an indirect land use model.

In [78]:
def remove_iluc_land(data):
    EXCLUDED = ('Arable land, as ha*year-eq. (linked)', 'Market for arable land')
    data = [ds for ds in data if ds['name'] not in EXCLUDED]
    for ds in data:
        ds['exchanges'] = [exc for exc in ds['exchanges'] if exc.get('name') not in EXCLUDED]
    return data

In [79]:
ei.apply_strategy(remove_iluc_land)

Applying strategy: remove_iluc_land


This input data assumes the existence of electricity mixes in EXIOBASE, but they don't actually exist (I guess you could create them). We have three unlinked exchanges to `Electricity mix` in WA, WL, and WE, but luckily they *aren't used anywhere*, and so we can drop these exchanges without worry.

In [80]:
def remove_missing_electricity_market_exchanges(data):
    drop_me = lambda e: e.get('name') == 'Electricity mix' and e['location'] in ('WA', 'WL', 'WE')
    
    for ds in data:
        ds['exchanges'] = [exc for exc in ds['exchanges'] if not drop_me(exc)]
    return data

In [81]:
ei.apply_strategy(remove_missing_electricity_market_exchanges)

Applying strategy: remove_missing_electricity_market_exchanges


Just for laughs (!?), there are some exchanges where activities where some exchanges are linked in kilograms, and others in tons. So let's fix exchanges in kilograms and switch them to tons when that will produce a link.

In [82]:
def switch_exchange_units_when_helpful(data):
    lookup = {(ds['name'], ds['unit']) for ds in data}
    
    for ds in data:
        for exc in ds['exchanges']:
            if exc.get('input'):
                continue
            if exc['unit'] == 'kilogram' and (exc['name'], 'ton') in lookup:
                exc['unit'] = 'ton'
                exc['amount'] /= 1000
    return data

In [83]:
ei.apply_strategy(switch_exchange_units_when_helpful)

Applying strategy: switch_exchange_units_when_helpful


In [84]:
ei.match_database(fields=['name', 'location', 'unit'])

Applying strategy: link_iterable_by_fields


In [85]:
ei.statistics()

266 datasets
899 exchanges
0 unlinked exchanges
  


(266, 899, 0)

In [86]:
ei.write_database()